In [13]:
import json
import re


In [14]:
def load_json(json_file):
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

def save_jsonl(data, jsonl_file):
    with open(jsonl_file, 'w', encoding='utf-8') as file:
        for entry in data:
            json.dump(entry, file, ensure_ascii=False)
            file.write('\n')

def save_json(data, json_file):
    with open(json_file, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False)
        
def count_chinese_chars(jsonl_file):
    chinese_char_count = 0
    with open(jsonl_file, 'r', encoding='utf-8') as file:
        for line in file:
            json_obj = json.loads(line)
            for key, value in json_obj.items():
                if isinstance(value, str):
                    chinese_char_count += count_chinese_chars_in_string(value)
    return chinese_char_count

def count_chinese_chars_in_string(string):
    count = 0
    for char in string:
        if '\u4e00' <= char <= '\u9fff':
            count += 1
    return count

In [10]:
json_file = 'me_train.json'
new_json_file = 'KD_train.json'
new_jsonl_file = 'KD_train.jsonl'

data = load_json(json_file)
json_no = list(data.keys())
data_clean = []
for q in json_no:
    question = data[q]['question']
    answer_dic = data[q]['evidences']
    for i in range(len(answer_dic.keys())):
        if i < 10:
            no = '#0'
        else:
            no = '#'
        answer = answer_dic[q+no+str(i)]['answer']
        evidence = answer_dic[q+no+str(i)]['evidence']

        if any(re.search(answer_single, evidence) for answer_single in answer):
            tmp = {}
            tmp["question"] = question
            tmp["answer"] = answer
            tmp["evidence"] = evidence
            data_clean.append(tmp)

save_jsonl(data_clean, new_jsonl_file)
save_json(data_clean, new_json_file)

In [11]:
data = load_json('KD_train.json')

In [12]:
len(data)

140878

In [17]:
jsonl_file = 'KD_test.jsonl'
chinese_chars_count = count_chinese_chars(jsonl_file)
print("汉字数：", chinese_chars_count)

汉字数： 3313226


In [ ]:
# 知识岛目前的数据集
1、餐饮行业：

a、餐饮行业8000问，来源：2022年和2023年餐饮行业报告与行业白皮书，由chatGPT 3.5负责整理总结，共8204条问答对，1.3M的tokens总数（680k汉字总数）

2、百度知道，来源：百度知道的帖子（2017年的数据整理），共211741条问答对，1.3M的tokens总数（680k汉字总数），42M的tokens总数（21M汉字总数），由于训练数据超过git的文件上限（25mb），因此可在hugging face上查看：https://huggingface.co/datasets/LIUshu123/knowledgeDAO/tree/main/WebQA